In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

# ✅ 데이터 불러오기
file_path = '/mnt/data/연간_급식_데이터.csv'
data = pd.read_csv(file_path, encoding='cp949')

# ✅ 독립변수 및 종속변수 분리
X = data[['선호도', '기온', '계절', '1학년_남', '1학년_여', '2학년_남', '2학년_여', '3학년_남', '3학년_여', '행사']]
y = data[['잔반량']]

# ✅ 범주형 변수 원핫 인코딩
X_encoded = pd.get_dummies(X, columns=['계절', '행사'])

# ✅ 수치형 변수 정규화
scaler = StandardScaler()
num_cols = ['선호도', '기온', '1학년_남', '1학년_여', '2학년_남', '2학년_여', '3학년_남', '3학년_여']
X_encoded[num_cols] = scaler.fit_transform(X_encoded[num_cols])

# ✅ 학습 / 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# ✅ 모델 정의 (MLP)
input_layer = tf.keras.layers.Input(shape=(X_encoded.shape[1],))
x = tf.keras.layers.Dense(32, activation='relu')(input_layer)
x = tf.keras.layers.Dense(16, activation='relu')(x)
output_layer = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:

# ✅ 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=10, batch_size=16)

In [ ]:
# ✅ 테스트셋 예측 결과 확인
print("예측 결과 (테스트셋 일부):")
print(model.predict(X_test[:5]))
print("실제 잔반량:")
print(y_test[:5].values)

# ✅ 새 데이터 예측 예시
new_input = pd.DataFrame([{
    '선호도': 4.5,
    '기온': 25,
    '계절': '여름',
    '1학년_남': 50,
    '1학년_여': 45,
    '2학년_남': 48,
    '2학년_여': 50,
    '3학년_남': 47,
    '3학년_여': 46,
    '행사': '없음'
}])

# 새 데이터 전처리
new_encoded = pd.get_dummies(new_input)
new_encoded = new_encoded.reindex(columns=X_encoded.columns, fill_value=0)
new_encoded[num_cols] = scaler.transform(new_encoded[num_cols])

# 새 데이터 예측
predicted_leftover = model.predict(new_encoded)
print("예측된 잔반량:", predicted_leftover[0][0])

# ✅ 모델 저장 및 로딩
model.save("leftover_prediction_model.keras")
reconstructed_model = keras.models.load_model("leftover_prediction_model.keras")